In [1]:
import carla
import numpy as np
import cv2
import time
import sys
sys.path.append('F:\CARLA\Windows\CARLA_0.9.15\PythonAPI\carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [2]:
client = carla.Client('localhost', 2000)

world = client.get_world()

blueprints = [bp for bp in world.get_blueprint_library().filter('*')]
for blueprint in blueprints:
   #print(blueprint.id)
   for attr in blueprint:
       #print('  - {}'.format(attr))
       None

In [3]:
def DrawPointsFor30Sec(world, spawn_points):
    drawn_points = []
    for index, waypoint in enumerate(spawn_points):
        # Draw a string with an ID at the location of each spawn point
        point_id = f'ID: {index}'
        point = world.debug.draw_string(
            waypoint.location,
            point_id,
            draw_shadow=False,
            color=carla.Color(r=255, g=255, b=255),
            life_time=30,  # Set to 0 to make it persist indefinitely
            persistent_lines=True
        )
        drawn_points.append(point)

def autogo(actors: list):
    # Set each actor to autopilot   
    for actor in actors:
        if actor is not None:  # Check if the actor was successfully spawned
            actor.set_autopilot(True)


In [4]:
spawn_points = world.get_map().get_spawn_points()
DrawPointsFor30Sec(world, spawn_points)

In [5]:

vehicle_bp = world.get_blueprint_library().filter('*mini*')
bicycle_bp = world.get_blueprint_library().filter('*crossbike*')

vehicle_start_point = spawn_points[1]
bicycle_start_point = spawn_points[1]


bicycle = world.try_spawn_actor(bicycle_bp[0], bicycle_start_point)
bicyclepos = carla.Transform(bicycle_start_point.location + carla.Location(x=10, y=-6), carla.Rotation(yaw=90))
bicycle.set_transform(bicyclepos)
control = carla.VehicleControl(throttle=0.5)  # Adjust throttle to control speed


bicycle2 = world.try_spawn_actor(bicycle_bp[0], bicycle_start_point)
bicyclepos = carla.Transform(bicycle_start_point.location + carla.Location(x=10, y=3))
bicycle2.set_transform(bicyclepos)

bicycle3 = world.try_spawn_actor(bicycle_bp[0], bicycle_start_point)
bicyclepos = carla.Transform(bicycle_start_point.location + carla.Location(x=15))
bicycle3.set_transform(bicyclepos)

bicycle4 = world.try_spawn_actor(bicycle_bp[0], bicycle_start_point)
bicyclepos = carla.Transform(bicycle_start_point.location + carla.Location(x=20, y=-3))
bicycle4.set_transform(bicyclepos)

vehicle = world.try_spawn_actor(vehicle_bp[0], vehicle_start_point)
#actors = [vehicle, bicycle, bicycle2, bicycle3, bicycle4]

#autogo(actors)




In [15]:
import math
v = bicycle.get_velocity()
speed = round(3.6 * math.sqrt(v.x**2 + v.y**2+v.z**2), 0)
print(speed)

0.0


In [26]:
from ultralytics import YOLO
#setup the camera
CAMERA_POS_Z = 1.5 
CAMERA_POS_X = 0
CAMERA1_POS_Y = 0.5
CAMERA2_POS_Y = -0.5

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')


camera1_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X, y = CAMERA1_POS_Y))
camera1 = world.spawn_actor(camera_bp,camera1_init_trans,attach_to=vehicle)

camera2_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X, y = CAMERA2_POS_Y))
camera2 = world.spawn_actor(camera_bp,camera2_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera1_data = {'image': np.zeros((image_h,image_w,4))}
camera2_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera1.listen(lambda image: camera_callback(image,camera1_data))
camera2.listen(lambda image: camera_callback(image,camera2_data))
#vehicle.set_autopilot(True)
model = YOLO("best.pt")

In [27]:
from scipy.spatial import distance
import math

def match_bicycles_between_left_right(bicycles_left: list, bicycles_right: list):
    image_w = 640  # Image width
    fov = 90  # Field of view in degrees
    baseline = 1.0  # Baseline distance in meters
    focal_length = image_w / (2 * math.tan(math.radians(fov / 2)))  # Focal length in pixels

    
    y_threshold = 20  # pixels, adjust based on image scale
    matched_bicycles_with_distances = []

    for left_bicycle in bicycles_left:
        left_x, left_y = left_bicycle
        closest_bicycle = None
        min_dist = float('inf')

        for right_bicycle in bicycles_right:
            right_x, right_y = right_bicycle
            # Check if the y-coordinates are similar
            if abs(left_y - right_y) < y_threshold:
                # Calculate the distance (disparity)
                dist = abs(left_x - right_x)
                if dist < min_dist:
                    min_dist = dist
                    closest_bicycle = right_bicycle

        # If a match was found, calculate depth and add to list
        if closest_bicycle:
            right_x, _ = closest_bicycle
            disparity = abs(left_x - right_x)
            depth = (focal_length * baseline) / disparity if disparity != 0 else float('inf')
            matched_bicycles_with_distances.append((left_bicycle, depth))

    return matched_bicycles_with_distances

In [28]:
prev_time = time.time()

#autogo(actors)
#bicycle.apply_control(control)

last_distance = None
last_time = None

while True:
    frame1 = camera1_data['image']
    frame2 = camera2_data['image']

    frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGRA2GRAY)
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGRA2GRAY)
    # Convert RGB image from BGRA to BGR
    frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGRA2BGR)
    frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGRA2BGR)
    #computeDepthMapSGBM(frame1_gray, frame2_gray)
    # Run the object detection model on the RGB frame
    results_left = model(frame1)
    results_right = model(frame2)
    
    bicycles_left = []
    bicycles_right = []



    for result in results_left:
        for box in result.boxes:
            # Extract box coordinates and other details
            x1, y1, x2, y2 = box.xyxy[0]
            center_x = int((x1 + x2) / 2)  # x-center of the bicycle
            center_y = int((y1 + y2) / 2)  # y-center of the bicycle
            bicycles_left.append((center_x, center_y))
            conf = box.conf[0]            # Confidence score
            cls = box.cls[0]
            cv2.rectangle(frame1, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            cv2.putText(frame1, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            #break
              # Bounding box coordinates
    
    for result in results_right:
        for box in result.boxes:
            # Extract box coordinates and other details
            x1, y1, x2, y2 = box.xyxy[0]
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            bicycles_right.append((center_x, center_y))
            conf = box.conf[0]            # Confidence score
            cls = box.cls[0]
            cv2.rectangle(frame2, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            cv2.putText(frame2, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            #break
              # Bounding box coordinates

    matched_bicycles_with_distances = match_bicycles_between_left_right(bicycles_left, bicycles_right)
    # Display distance for each matched bicycle on the left frame
    i = 30
    for (left_bicycle, distance) in matched_bicycles_with_distances:
        left_x, left_y = left_bicycle
        distance_label = f"Distance: {distance:.2f}m"
        cv2.putText(frame1, distance_label, (left_x, left_y-i), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        i += 10
    
    # Calculate FPS
    current_time = time.time()
    fps = 1 / (current_time - prev_time)
    prev_time = current_time

    fps_label = f"FPS: {fps:.2f}"
    cv2.putText(frame1, fps_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)


    # Display the combined frame
    frame1 = cv2.resize(frame1, (1200, 800))
    cv2.imshow('Camera1',frame1)

    # Exit loop on 'q' key press
    if cv2.waitKey(1) == ord('q'):
        break


# Cleanup
cv2.destroyAllWindows()


0: 384x640 4 Bicycles, 279.2ms
Speed: 3.0ms preprocess, 279.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Bicycles, 249.6ms
Speed: 3.0ms preprocess, 249.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Bicycles, 90.7ms
Speed: 2.0ms preprocess, 90.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Bicycles, 93.2ms
Speed: 2.0ms preprocess, 93.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Bicycles, 90.3ms
Speed: 3.0ms preprocess, 90.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Bicycles, 86.7ms
Speed: 2.0ms preprocess, 86.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Bicycles, 88.7ms
Speed: 1.0ms preprocess, 88.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Bicycles, 89.7ms
Speed: 2.0ms preprocess, 89.7ms inference, 1.0ms postprocess per imag

In [24]:
#Destroying the existing things
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()